In [34]:
# !pip install scattertext

In [50]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import requests
import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liviaclarete/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/liviaclarete/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/liviaclarete/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Read the file into a Pandas DataFrame
url = 'https://raw.githubusercontent.com/kcfraser54/Hash-Table/main/ulyss12.txt'
response = requests.get(url)
text = response.text

In [266]:
import re
# tokenize sentences 
def replace_r_with_space(text):
    lines = text.split('\r\n')
    replaced_lines = []
    for line in lines:
        if '\r' in line:
            parts = line.split('\r')
            replaced_line = ' '.join(parts)
        else:
            replaced_line = line
        replaced_lines.append(replaced_line)
    replaced_text = ' '.join(replaced_lines)
    return replaced_text

def split_string(text):
    text = replace_r_with_space(text)
    pattern = r'[.?!\[\]{}<>|/\n]+'
    split_text = re.split(pattern, text)
    stripped_strings = [s.strip() for s in split_text]
    return stripped_strings


In [259]:
sentences = split_string(text)

In [262]:
nlp = spacy.load('en_core_web_sm')

def process_text_data(text_list):
    tokenized_sentences = []
    word_counts = []
    propn_counts = []
    noun_counts = []
    adj_counts = []
    verb_counts = []
    number_counts = []

    for text in text_list:
        doc = nlp(text)
        tokenized_sentences.append(text)
        word_counts.append(len(doc))
        propn_counts.append(len([token for token in doc if token.pos_ == 'PROPN']))
        noun_counts.append(len([token for token in doc if token.pos_ == 'NOUN']))
        adj_counts.append(len([token for token in doc if token.pos_ == 'ADJ']))
        verb_counts.append(len([token for token in doc if token.pos_ == 'VERB']))
        number_counts.append(len([token for token in doc if token.like_num]))

    data = {
        'text': tokenized_sentences,
        'words': word_counts,
        'PROPN': propn_counts,
        'NOUN': noun_counts,
        'ADJ': adj_counts,
        'VERB': verb_counts,
        'Numbers': number_counts
    }

    df = pd.DataFrame(data)
    return df

In [264]:
t = process_text_data(sentences)

In [265]:
t.head()

,text,words,PROPN,NOUN,ADJ,VERB,Numbers
0,"﻿ The Project Gutenberg EBook of Ulysses, by J...",30,7,3,0,0,0
1,"You may copy it, give it away or re-use it und...",29,4,2,0,6,0
2,gutenberg,1,1,0,0,0,0
3,org Title: Ulysses Author: James Joyce Rel...,18,7,2,0,0,2
4,EBook #4300,3,1,0,0,0,1


In [270]:
t = t.sort_values('words', ascending=False)

In [273]:
t.head()

,text,words,PROPN,NOUN,ADJ,VERB,Numbers
25329,- each and or let him pay it and invite some o...,5251,185,809,298,759,58
25324,6 per doz going out to see her aunt if you ple...,4402,106,583,193,701,36
25327,- in all sure you cant get on in this world wi...,4272,180,669,246,586,52
25328,Mulveys was the first when I was in bed that m...,2999,138,484,159,447,32
25331,2 a minute even if some of it went down what i...,2228,85,337,120,310,22


In [391]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download required NLTK resources
nltk.download('vader_lexicon')

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define the sentiment analysis function
def analyze_sentiment(text):
    # Perform sentiment analysis using VADER
    sentiment_scores = sia.polarity_scores(text)
    
    # Extract the compound score
    compound_score = sentiment_scores['compound']
    
    # Classify the sentiment based on the compound score
    if compound_score >= 0.05:
        sentiment = 'Positive'
    elif compound_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    
    return sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/liviaclarete/nltk_data...


In [397]:
t['sentiment'] = t.text.apply(analyze_sentiment)

In [408]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=6)

def analyze_emotion(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        logits = model(**encoded_input).logits
    probabilities = torch.softmax(logits, dim=1).tolist()[0]
    emotion_labels = ['anger', 'joy', 'optimism', 'sadness', 'surprise', 'trust']
    emotions = {label: prob for label, prob in zip(emotion_labels, probabilities)}
    most_important_emotion = max(emotions, key=emotions.get)
    return most_important_emotion

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [405]:
t['emotion'] = t.text.apply(analyze_emotion)

In [406]:
t.head()

,text,words,PROPN,NOUN,ADJ,VERB,Numbers,sentiment,emotion
25329,- each and or let him pay it and invite some o...,5251,185,809,298,759,58,Positive,"{'anger': 0.12793628871440887, 'joy': 0.160932..."
25324,6 per doz going out to see her aunt if you ple...,4402,106,583,193,701,36,Positive,"{'anger': 0.12212774902582169, 'joy': 0.153212..."
25327,- in all sure you cant get on in this world wi...,4272,180,669,246,586,52,Positive,"{'anger': 0.1349671632051468, 'joy': 0.1564356..."
25328,Mulveys was the first when I was in bed that m...,2999,138,484,159,447,32,Positive,"{'anger': 0.10962546616792679, 'joy': 0.211760..."
25331,2 a minute even if some of it went down what i...,2228,85,337,120,310,22,Positive,"{'anger': 0.1235189139842987, 'joy': 0.1550717..."


In [417]:
import scattertext as st

movie_df = st.SampleCorpora.RottenTomatoes.get_data()

corpus = st.CorpusFromPandas(
    t,
    category_col='sentiment',
    text_col='text',
    nlp=st.whitespace_nlp_with_sentences
).build().get_unigram_corpus()

beta_posterior = st.BetaPosterior(corpus).set_categories('Positive', ['Negative'])
score_df = beta_posterior.get_score_df()
print("Top Fresh Terms")
print(score_df.sort_values(by='cat_p').head())

print("Top Rotten Terms")
print(score_df.sort_values(by='ncat_p').head())

html = st.produce_frequency_explorer(
    corpus,
    category='Positive',
    not_category_name='Negative',
    term_scorer=beta_posterior,
    grey_threshold=1
)

file_name = 'ulysses_sentiment_visualization.html'
open(file_name, 'wb').write(html.encode('utf-8'))

/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_freq_df['score'][cat_freq_df['cat_pct'] == cat_freq_df['ncat_pct']] = 0
/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_freq_df['score'][cat_freq_df['cat_pct'] < cat_freq_df['ncat_pct']] = cat_freq_df['ncat_z']
/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:90: SettingWithCopyWarning: 
A value 

Top Fresh Terms
      cat  ncat  all   cat_pct  ncat_pct         cat_p  ncat_p     cat_z   
term                                                                       
hand  270    16  286  0.003172  0.000312  2.277157e-14     1.0 -7.544097  \
like  426    72  498  0.005005  0.001405  4.533541e-14     1.0 -7.453827   
good  261    21  282  0.003067  0.000410  5.877961e-13     1.0 -7.108208   
yes   186    10  196  0.002185  0.000195  1.195054e-10     1.0 -6.333919   
best  122     4  126  0.001433  0.000078  4.427536e-08     1.0 -5.348777   

      ncat_z     score  
term                    
hand     inf  7.544097  
like     inf  7.453827  
good     inf  7.108208  
yes      inf  6.333919  
best     inf  5.348777  
Top Rotten Terms
        cat  ncat  all   cat_pct  ncat_pct  cat_p        ncat_p  cat_z   
term                                                                     
no      119   421  540  0.001398  0.008216    1.0  6.377762e-84    inf  \
poor     23   134  157  0.000270  0.0

/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_freq_df['score'][cat_freq_df['cat_pct'] == cat_freq_df['ncat_pct']] = 0
/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_freq_df['score'][cat_freq_df['cat_pct'] < cat_freq_df['ncat_pct']] = cat_freq_df['ncat_z']
/Users/liviaclarete/.virtualenvs/nlp/lib/python3.9/site-packages/scattertext/termscoring/BetaPosterior.py:90: SettingWithCopyWarning: 
A value 

2912850

In [331]:
def count_words(text, chunk_size=100000):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation.replace('.', '')))

    # Convert text to lowercase
    text = text.lower()

    # Filter only numbers and words
    filtered_text = re.findall(r'\b[a-zA-Z0-9]+\b', text)

    # Tokenize the text into words
    words = word_tokenize(' '.join(filtered_text))

    # Remove stopwords
    stopwords_set = set(stopwords.words('english'))
    words = [word for word in words if word not in stopwords_set]

    # Initialize spaCy
    nlp = spacy.load('en_core_web_lg')

    # Process text in chunks
    lemmatized_words = []
    token_types = []
    for i in range(0, len(words), chunk_size):
        chunk = words[i:i+chunk_size]
        chunk_text = ' '.join(chunk)

        # Lemmatize chunk using spaCy
        doc = nlp(chunk_text)
        chunk_lemmas = [token.lemma_ for token in doc]
        lemmatized_words.extend(chunk_lemmas)
        token_types.extend([token.pos_ for token in doc])

    # Count the occurrence of each word
    word_counts = {}
    for word, token_type in zip(lemmatized_words, token_types):
        if len(word) > 2:
            if word in word_counts:
                word_counts[word][0] += 1
            else:
                word_counts[word] = [1, token_type]

    # Create a DataFrame from the word counts
    df = pd.DataFrame.from_dict(word_counts, orient='index', columns=['count_words', 'token_type'])
    df.index.name = 'words'
    df.reset_index(inplace=True)
    df = df.sort_values(by='count_words', ascending=False)

    return df

df_counts = count_words(text)

df_counts.head(10)

,words,count_words,token_type
138,say,1993,VERB
3640,bloom,1007,PROPN
130,like,776,ADP
157,one,725,NUM
427,see,704,VERB
54,come,640,VERB
298,man,598,NOUN
104,stephen,572,PROPN
280,know,553,VERB
183,old,505,ADJ


In [418]:
df_counts.token_type.value_counts()

token_type
PROPN    7748
NOUN     7683
ADJ      4092
VERB     3066
ADV      1072
NUM       145
X          99
AUX        83
ADP        66
PRON       37
INTJ       30
SCONJ      22
DET        17
CCONJ      10
PART        1
PUNCT       1
Name: count, dtype: int64